In [1]:
using Random, Distributions, LinearAlgebra, FunctionOperators, ToeplitzMatrices, Formatting

# Robust Principal Component Analysis?
*by Emmanuel J. Candès, Xiaodong Li, Yi Ma, and John Wright*  
https://arxiv.org/pdf/0912.3599.pdf

## 4.1 Exact recovery from varying fractions of error

[...] We consider square matrices of varying dimension $n = 500, \ldots , 3000$. We generate a rank-$r$ matrix $L_0$ as a product $L_0 = XY^∗$ where $X$ and $Y$ are $n \times r$ matrices with entries independently sampled from a $\mathcal{N}(0,1/n)$ distribution. $S_0$ is generated by choosing a support set $\Omega$ of size $k$ uniformly at random, and setting $S_0 = \mathcal{P}_\Omega E$, where $E$ is a matrix with independent Bernoulli $\pm 1$ entries. [...]

In [2]:
function generateLowRankComponent_Candes(n, r, dType)
    𝓝 = Normal(0, 1/n)
    X = rand(𝓝, dType, n, r)
    Y = rand(𝓝, dType, n, r)
    L₀ = X * Y'
end

generateLowRankComponent_Candes (generic function with 1 method)

In [3]:
function generateSparseComponent_Candes(n, k, dType)
    Ω = randperm(n*n)[1:k] # indices of non-zero elements
    𝓑 = Binomial()
    E = rand(𝓑, n, n)
    E[E.==0] .= -1
    S₀ = zeros(n, n)
    S₀[Ω] = E[Ω]
    S₀
end

generateSparseComponent_Candes (generic function with 1 method)

# Christian's Code

This function randomly samples a $(d_1 \times d_2)$ sparse matrix with ones at $m$ randomly chosen
coordinates (uniform without replacement). The output matrix has at least $r$ non-zero entries
in each row and each column, where $r$ is a specified positive integer. The number of ones in the
output matrix is exactly $m$.

In [4]:
function generateΦ(d₁, d₂, r, m)
    @assert max(d₁, d₂) * r ≤ m
    @assert m ≤ d₁ * d₂
    @assert r ≤ d₁
    @assert r ≤ d₂
    
    # generate a square matrix where each row and each column has exactly r ones
    initial = Circulant([fill(1, r)..., fill(0, min(d₁, d₂) - r)...])
    
    # Extend that matrix to a d₁×d₂ matrix where each row and each column has at least r ones
    # That is accomplished by repeating the "initial" matrix and then cropping
    if d₁ < d₂
        M = repeat(initial, outer = (1, ceil(Int, d₂ / d₁)))
    elseif d₁ > d₂
        M = repeat(initial, outer = (ceil(Int, d₁ / d₂), 1))
    else
        M = initial
    end
    M = M[1:d₁, 1:d₂]
    
    # Randomly switch zeros to ones until exactly m number of ones are in the matrix
    zero_places = findall(M .== 0)
    number_of_missing_ones = m - (d₁*d₂ - length(zero_places))
    number_of_missing_ones > 0 && (M[shuffle(zero_places)[1:number_of_missing_ones]] .= 1)
    
    # Then randomize matrix by permutating rows and columns a couple times
    for i in 1:10
        M .= M[shuffle(1:end), :] # shuffle rows
        M .= M[:, shuffle(1:end)] # shuffle columns
    end
    
    M
end

generateΦ (generic function with 1 method)

Transform $\Phi^M$ to $\Phi$ such that $\Phi^M \circ \mathbf{M} = \Phi \mathbf{M}_{vec}$, where $\circ$ is the Hadamard product (element-wise multiplication) between two matrices, and $\cdot_{vec}$ is the vectorization operator.

In [5]:
function HadamardProd_to_MatrixMult(Φᴹ)
    m = convert(Int, sum(Φᴹ))
    d₁, d₂ = size(Φᴹ)

    Φ = zeros(m, length(Φᴹ))
    non_zero_places = findall(vec(Φᴹ) .== 1)
    for i in 1:m
        Φ[i, non_zero_places[i]] = 1
    end
    return Φ
end

HadamardProd_to_MatrixMult (generic function with 1 method)

```julia
generateLowRankComponent_Christian(d₁::Int, d₂::Int, r::Int, dType::Type;
                                    S_mode::String = "ones", condition_number::Int = 1)
```

Constructs a random rank-$r$ matrix $X_0$ of dimension $(d_1 \times d_2)$ and
with element type `dType`.

The randomness comes from Gaussian matrices, parametrized such that $X_0 = U_0 * S_0 * V_0^*$
where $U_0 \in \mathbb{R}^{d_1 \times r}$ and $V_0 \in \mathbb{R}^{d_2 \times r}$ are
matrices with i.i.d. standard Gaussian entries and $S_0$ is a diagonal matrix whose elements
are specified by `S_mode` parameter:
 - `"ones"`: $S_0$ is an identity matrix
 - `"normal"`: Diagonal entries of $S_0$ are i.i.d. standard Gaussian entries
 - `"condition_control_linear"`: $S_{1,1} = $ `condition_number`, $S_{r,r} = 1$, with _linear_ interpolation in between.
 - `"condition_control_1/x²"`: $S_{1,1} = $ `condition_number`, $S_{r,r} = 1$, with interpolation of the order (1/x^2) in between.
 - `"condition_control_log"`: $S_{1,1} = $ `condition_number`, $S_{r,r} = 1$, with  _exponential_ interpolation in between.

Corresponding Matlab function: https://github.com/ckuemmerle/MatrixIRLS/blob/master/tools/sample_phi_MatrixCompletion.m

In [6]:
function generateLowRankComponent_Christian(d₁::Int, d₂::Int, r::Int, dType::Type;
        S_mode::String = "ones", condition_number::Int = 1)
    𝓝 = Normal(0, 1)
    U, V = rand(𝓝, dType, d₁, r), rand(𝓝, dType, d₂, r)
    if S_mode == "ones"
        S = Diagonal(ones(real(dType), r))
    elseif S_mode == "normal"
        S = Diagonal(rand(𝓝, real(dType), r))
    elseif S_mode == "condition_control_linear"
        S = Diagonal(range(condition_number, 1, length=r))
    elseif S_mode == "condition_control_1/x²"
        r² = r^2
        scale = condition_number - (1 - condition_number/r²)/(1 - 1/r²)
        shift = (1 - condition_number/r²) / (1 - 1/r²)
        S = Diagonal(scale ./ collect(1:r).^2 .+ shift)
    elseif S_mode == "condition_control_log"
        x = -log(condition_number) .* (collect(1:r) .- 1) ./ (r - 1)
        S = Diagonal(condition_number .* exp.(x))
    else
        throw(ArgumentError("S_mode $S_mode is not known"))
    end
    U * S * V'
end

generateLowRankComponent_Christian (generic function with 1 method)

## Other helper functions

Get input/output size of a `FunctionOperator`.

In [7]:
import Base.size
function Base.size(FO::FunctionOperator, d::Int)
    @assert d in [1, 2]
    prod(d == 1 ? FO.outDims : FO.inDims)
end

Extend functionality of built-in function `rand`: sampling from a given Normal distribution.

In [8]:
import Random.rand
rand(N::Normal, T::Type{X}, dims...) where {X<:AbstractFloat} = T.(rand(N,  dims...))
rand(N::Normal, T::Type{Complex{X}}, dims...) where {X} =
    (1/√2) * (Complex{X}.(rand(N, dims...)) + Complex{X}.(rand(N, dims...))im)

rand (generic function with 167 methods)

Print iteration info in a fancy table.

In [9]:
module DebugTableModule

import Formatting
export DebugTable, printRow, finishTable

mutable struct DebugTable
    columns::Vector{Union{Tuple{String,Function}, Tuple{String,Function,Int},
                    Tuple{String,Nothing}, Tuple{String,Nothing,Int}}}
    defaultWidth::Int64
    defaultPrecision::Int64
    headAlreadyPrinted::Bool
end

DebugTable(tuples...; defaultWidth = 8, defaultPrecision = 3, headAlreadyPrinted = false) =
    DebugTable([e for e in tuples], defaultWidth, defaultPrecision, headAlreadyPrinted)

processColumnWidth(columns, defaultWidth) = begin
    for (i,columnTuple) in enumerate(columns)
        if length(columnTuple) < 3
            columns[i] = columnTuple..., max(length(columnTuple[1]), defaultWidth)
        elseif length(columnTuple[1]) > columnTuple[3]
            columns[i] = columnTuple[1], columnTuple[2], length(columnTuple[1])
        end
    end
    columns
end

printLine(columns, first, middle, last) = begin
    print(first)
    for (i,(_,_,cnw)) in enumerate(columns)
        print(repeat("─", cnw + 2))
        i < length(columns) && print(middle)
    end
    println(last)
end

center(str, width) = repeat(" ", ceil(Int, (width - length(str))/2)) * str *
    repeat(" ", floor(Int, (width - length(str))/2))

printColumnNames(columns) = begin
    for columnTuple in columns
        print("│ ", center(columnTuple[1], columnTuple[3]), " ")
    end
    println("│")
end

printHeader(DT::DebugTable) = begin
    processColumnWidth(DT.columns, DT.defaultWidth)
    printLine(DT.columns, "┌", "┬", "┐")
    printColumnNames(DT.columns)
    printLine(DT.columns, "├", "┼", "┤")
    DT.headAlreadyPrinted = true
end

printValue(value::Int, width, precision) =
    abs(value > 10^width) ?
        printValue(float(value), width, precision) : 
        Formatting.sprintf1("%$(width)d", value)
printValue(value::Real, width, precision) =
    Formatting.sprintf1(abs(value) < 10. ^ -(precision-1) || abs(value) > 10^(width-precision-1) ?
            "%$width.$(precision-1)e" : "%$width.$(precision)f", value)

function printRow(DT::DebugTable, args...; last::Bool = false)
    DT.headAlreadyPrinted || printHeader(DT)
    for (name, expr, width) in DT.columns
        positionInArgs = findfirst(x -> x[1] == name, args)
        value = positionInArgs !== nothing ? args[positionInArgs][2] : expr()
        print("│ ", printValue(value, width, DT.defaultPrecision), " ")
    end
    println("│")
    last && printLine(DT.columns, "└", "┴", "┘")
end

function finishTable(DT::DebugTable)
    printLine(DT.columns, "└", "┴", "┘")
end

end

Main.DebugTableModule